# 【課題】高エネルギー実験で生成された荷電粒子の飛跡を見つける

必要なライブラリを最初にインポートします。

In [ ]:
import os
import sys

repo_dir = os.path.join(os.environ['HOME'], 'qc-workbook-lecturenotes')
sys.path.append(repo_dir)

if not os.path.exists(os.path.join(repo_dir, 'qc_workbook', 'hepqpr')):
    import subprocess
    
    proc = subprocess.Popen(['git', 'submodule', 'init'], cwd=repo_dir)
    proc.wait()
    
    proc = subprocess.Popen(['git', 'submodule', 'update'], cwd=repo_dir)
    proc.wait()
    
    os.symlink(os.path.join(repo_dir, 'hepqpr-qallse', 'src', 'hepqpr'), os.path.join(repo_dir, 'qc_workbook', 'hepqpr'))
    
    %pip install 'git+https://github.com/LAL/trackml-library.git'
    
    %pip install dwave-qbsolv    

In [ ]:
# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import pprint
import numpy as np
import h5py
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator
from qiskit.algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.algorithms.gradients import ParamShiftEstimatorGradient
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_optimization.applications import OptimizationApplication
from qiskit_aer import AerSimulator

from collections import OrderedDict
from qiskit.opflow import StateFn
def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    elif isinstance(state_vector, StateFn):
        binary_string = list(state_vector.sample().keys())[0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x

### ハミルトニアンの構成とVQEの実行

#### QUBO

ワークブックにあるセットアップで、各セグメントを粒子飛跡の一部として採用するかフェイクとして棄却するかを考えます。具体的には、$N$個のセグメントのうち$i$番目の採用・棄却を二値変数$T_i$の値1と0に対応させ、目的関数

$$
O(b, T) = \sum_{i=1}^N a_{i} T_i + \sum_{i=1}^N \sum_{j<i}^N b_{ij} T_i T_j
$$

を最小化する$\{T_i\}$を求めます。ここで$a_i$は上で決めたセグメント$i$のスコア、$b_{ij}$はセグメント$i$と$j$のペアのスコアです。$a_i$の値が小さい（検出器中心を向いている）、かつ$b_{ij}$の値が小さい（正しい飛跡と無矛盾な）ペアを組んでいるセグメントを採用し、そうでないものを棄却するほど、$O$の値は小さくなります。採用すべきセグメントが決まれば、それに基づいてすべての粒子飛跡を再構成できるので、この最小化問題を解くことがトラッキングに対応します。

それでは、まずスコア$a_{i}$と$b_{ij}$を読み出しましょう。この後のプログラミングを簡単にするため、二つのスコアセットは一つの二次元配列`b`に記録されており、`b[i, i]`が$a_i$に対応するようになっています。

In [ ]:
# スコアの読み込み
with h5py.File('data/QUBO_05pct_input.h5', 'r') as source:
    a_score = source['a_score'][()]
    b_score = source['b_score'][()]

print(f'Number of segments: {a_score.shape[0]}')
# 最初の5x5をプリント
print(a_score[:5])
print(b_score[:5, :5])

#### Ising形式

QUBOの目的関数はまだハミルトニアンの形になっていない（エルミート演算子でない）ので、VQEを使ってこの問題を解くにはさらに問題を変形する必要があります。ここで$T_i$が$\{0, 1\}$のバイナリー値を持つことに着目すると、

$$
T_i = \frac{1}{2} (1 - s_i)
$$

で値$\{+1, -1\}$を持つ変数$s_i$を定義できます。次に、$\{+1, -1\}$はパウリ演算子の固有値でもあるため、$s_i$を量子ビット$i$にかかるパウリ$Z$演算子で置き換えると、$N$量子ビット系の各計算基底がセグメントの採用・棄却をエンコードする固有状態となるような目的ハミルトニアン

$$
H(h, J, s) = \sum_{i=1}^N h_i Z_i + \sum_{i=1}^N \sum_{j<i}^N J_{ij} Z_i Z_j + \text{(constant)}
$$

が得られます。これは物理を始め自然科学の様々な場面で登場するIsing模型のハミルトニアンと同じ形になっています。右辺の$constant$はハミルトニアンの定数項で、変分法において意味を持たないので以降は無視します。

### 問題

以下のセルで、上の処方に従ってIsingハミルトニアンの係数$h_i$と$J_{ij}$を計算してください。

In [ ]:
num_qubits = a_score.shape[0]

coeff_h = np.zeros(num_qubits)
coeff_J = np.zeros((num_qubits, num_qubits))

##################
### EDIT BELOW ###
##################

# coeff_hとcoeff_Jをb_ijから計算してください

##################
### EDIT ABOVE ###
##################

次に、この係数をもとに、VQEに渡すハミルトニアンをSparsePauliOpとして定義します。{ref}`vqe_imp`ではSparsePauliOpは単一のパウリ積$ZXY$を表現するのに使いましたが、実はパウリ積の和も同じクラスを使って表現できます。例えば

$$
H = 0.2 IIZ + 0.3 ZZI + 0.1 ZIZ
$$

は

```python
H = SparsePauliOp(['IIZ', 'ZZI', 'ZIZ'], coeffs=[0.2, 0.3, 0.1])
```

となります。このとき、通常のQiskitの約束に従って、量子ビットの順番が右から左（一番右が第0量子ビットにかかる演算子）であることに注意してください。

### 問題

以下のセルで、 係数が0でないパウリ積をすべて拾い出し、対応する係数の配列を作成してください。

In [ ]:
##################
### EDIT BELOW ###
##################

# 係数が0でないパウリ積をすべて拾い出し、対応する係数の配列を作成してください

pauli_products = []
coeffs = []

##################
### EDIT ABOVE ###
##################

hamiltonian = SparsePauliOp(pauli_products, coeffs=coeffs)

### VQEの実行

In [ ]:
# ハミルトニアン行列を対角化して、エネルギーの最小固有値と固有ベクトルを求める
ee = NumPyMinimumEigensolver()
result_diag = ee.compute_minimum_eigenvalue(hamiltonian)

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print(f'Minimum eigenvalue (diagonalization): {result_diag.eigenvalue.real}')
# 解状態を計算基底で展開し、最も確率の高い計算基底を選ぶ
optimal_segments_diag = OptimizationApplication.sample_most_likely(result_diag.eigenstate)
print(f'Optimal segments (diagonalization): {optimal_segments_diag}')

In [ ]:
backend = AerSimulator()
# Estimatorインスタンスを作る
estimator = BackendEstimator(backend)

# VQE用の変分フォームを定義。ここではTwoLocalという組み込み関数を使う
ansatz = TwoLocal(num_qubits, 'ry', 'cz', 'linear', reps=1)

# オプティマイザーを選ぶ
optimizer_name = 'SPSA'

if optimizer_name == 'SPSA':
    optimizer = SPSA(maxiter=300)
    grad = ParamShiftEstimatorGradient(estimator)

elif optimizer_name == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
    grad = None

# パラメータの初期値をランダムに設定
rng = np.random.default_rng()
init = rng.uniform(0., 2. * np.pi, size=len(ansatz.parameters))

# VQEオブジェクトを作り、基底状態を探索する
vqe = VQE(estimator, ansatz, optimizer, gradient=grad, initial_point=init)
result_vqe = vqe.compute_minimum_eigenvalue(hamiltonian)

# 最適解のパラメータ値をansatzに代入し、状態ベクトルを計算する
optimal_state = Statevector(ansatz.bind_parameters(result_vqe.optimal_parameters))

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print(f'Minimum eigenvalue (VQE): {result_vqe.eigenvalue.real}')
optimal_segments_vqe = OptimizationApplication.sample_most_likely(optimal_state)
print(f'Optimal segments (VQE): {optimal_segments_vqe}')

### おまけ

Trackingがうまく行っても、この答えだと0と1が並んでいるだけで面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。

うまく処理が終われば、いくつかの情報とともに"tracks found: 1"という結果が出て、その時の飛跡の図（plot-ising_found_tracks.html）が左のサイドバーに作られていると思います。この図は、QUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面に投影して、どのヒットが選ばれたかを分かりやすく可視化したものです。緑の線が実際に見つかった飛跡で、青の線を含めたものが全体の飛跡の候補です。この実習では限られた数の量子ビットしか使っていないため、大部分の飛跡は見つけられていませんが、計算に使った3点ヒットから正しく飛跡が1本見つかっていることが分かると思います。

In [ ]:
from hepqpr.qallse import DataWrapper, Qallse, TrackRecreaterD
from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
from hepqpr.qallse.utils import diff_rows

optimal_segments = optimal_segments_vqe
# optimal_segments = optimal_segments_diag

# セグメントにはそれぞれIDがついているので、{ID: 0 or 1}の形でQallseにデータを渡す
with h5py.File('data/QUBO_05pct_input.h5', 'r') as source:
    triplet_keys = map(lambda key: key.decode('UTF-8'), source['triplet_keys'][()])

samples = dict(zip(triplet_keys, optimal_segments))

# get the results
all_doublets = Qallse.process_sample(samples)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

dw = DataWrapper.from_path('data/event000001000-hits.csv')

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)

**提出するもの**
- ハミルトニアンを実装する部分のコード